# Functions


In [ ]:
import dataclasses
from datetime import date


@dataclasses.dataclass
class RehearsalSlot:
    """Represents a single, structured rehearsal event."""
    rehearsal_date: date
    day_of_week: str  # Standardized, lowercase e.g., 'wednesday'
    start_time: int   # Military time, e.g., 1100
    end_time: int     # Military time, e.g., 1200


In [ ]:
import dataclasses
from abc import ABC, abstractmethod
from typing import TypeAlias


TimeInterval: TypeAlias = tuple[int, int]

class UnavailabilityConstraint(ABC):
    """Abstract base class for all dancer unavailability constraints."""
    @abstractmethod
    def get_conflicting_intervals(self, slot: RehearsalSlot) -> list[TimeInterval]:
        pass

@dataclasses.dataclass(frozen=True)
class DayOfWeekConstraint(UnavailabilityConstraint):
    """Represents an unavailability for an entire day of the week."""
    day_of_week: str

    def get_conflicting_intervals(self, slot: RehearsalSlot) -> list[TimeInterval]:
        if self.day_of_week == slot.day_of_week:
            return [(slot.start_time, slot.end_time)]
        return []


@dataclasses.dataclass(frozen=True)
class TimeOnDayConstraint(UnavailabilityConstraint):
    """
    Represents an unavailability for a specific time interval on a given day of the week.
    e.g., "Mondays from 900 to 1200"
    """
    day_of_week: str  # e.g., 'monday'
    start_time: int   # Military time, e.g., 900
    end_time: int     # Military time, e.g., 1200

    def get_conflicting_intervals(self, slot: RehearsalSlot) -> list[TimeInterval]:
        """
        Checks for conflicts if the day matches AND the time intervals overlap.
        """
        if self.day_of_week != slot.day_of_week:
            return [] # No conflict if it's not on the right day.

        # Classic interval overlap check: max(starts) < min(ends)
        overlap_start = max(self.start_time, slot.start_time)
        overlap_end = min(self.end_time, slot.end_time)

        if overlap_start < overlap_end:
            # A conflict exists! Return the actual interval of the conflict.
            return [(overlap_start, overlap_end)]
        
        return []

In [ ]:
def check_for_conflict(
    rehearsal: RehearsalSlot,
    dancer_constraints: list[UnavailabilityConstraint]
) -> list[TimeInterval]:
    """Checks for conflicts and returns a list of conflicting intervals."""
    all_conflicts = []
    for constraint in dancer_constraints:
        # print(f"constraint is {constraint}")
        conflicting_intervals = constraint.get_conflicting_intervals(rehearsal)
        all_conflicts.extend(conflicting_intervals)
    return all_conflicts

In [ ]:
from lark import Lark
from lark.exceptions import LarkError
from typing import List, Optional, Tuple

# Assuming your SemanticValidationError is defined as before
class SemanticValidationError(ValueError):
    """Raised when the syntax is valid but the meaning is not."""
    pass

def validate_and_parse_all_constraints(parser: Lark, text: str) -> Tuple[bool, Optional[List], Optional[str]]:
    """
    Parses a comma-separated string of constraints, validating each one individually
    and collecting ALL errors found.

    Returns:
        A tuple of (success, results, error_message).
        - If ALL constraints are valid, (True, [list_of_constraints], None).
        - If ANY constraint is invalid, (False, None, "multi-line error report").
    """
    parsed_constraints = []
    error_messages = []

    constraint_chunks = [chunk.strip() for chunk in text.split(',') if chunk.strip()]
    
    if not constraint_chunks:
        return (True, [], None)

    for chunk in constraint_chunks:
        try:
            parsed_result = parser.parse(chunk)
            
            # THE FIX: Use .append() to add the single object returned by the parser.
            # .extend() would fail with a TypeError because the parsed_result is not a list.
            parsed_constraints.append(parsed_result)

        except LarkError:
            msg = f"- Constraint '{chunk}' is not grammatically valid."
            error_messages.append(msg)
        
        except SemanticValidationError as e:
            msg = f"- Constraint '{chunk}' is not logically valid: {e}"
            error_messages.append(msg)
            
        except Exception as e:
            # This block will now only catch truly unexpected errors.
            msg = f"- An unexpected error occurred on constraint '{chunk}': {e}"
            error_messages.append(msg)

    if error_messages:
        header = "Found one or more invalid constraints:"
        # Corrected usage of '\n' for newlines
        final_error_message = f"{header}\n" + "\n".join(error_messages)
        return (False, None, final_error_message)
    else:
        return (True, parsed_constraints, None)

## Grammar Specification

In [ ]:
from lark import Lark, Transformer, v_args
from typing import TypeAlias, Optional

# --- A custom exception for semantic errors ---
class SemanticValidationError(ValueError):
    """Raised when the syntax is valid but the meaning is not."""
    pass


## executable code

In [1]:
from dance_scheduler.grammar import constraint_parser
from dance_scheduler.constraints import DayOfWeekConstraint, TimeOnDayConstraint

expected_output = [TimeOnDayConstraint(day_of_week='sunday', start_time=1200, end_time=2359)]
parser = constraint_parser(debug=True)
result = parser.parse("f after 10")
# result = parser.parse("sun before 500")
print()
print(f" result = {result}")
# expected_output = [TimeOnDayConstraint(day_of_week='sunday', start_time=Tree('normalize_time', [Token('INT', '12'), Token('AM_PM', 'pm')]), end_time=2359)]
assert result == expected_output

GrammarError: Rule 'time_range_tuple' used but not defined (in rule time_on_day_spec)

## Conflicts

In [ ]:
rehearsal_1 = RehearsalSlot(date(2024, 9, 4), 'wednesday', 1800, 1930) # Should conflict
rehearsal_2 = RehearsalSlot(date(2024, 9, 5), 'thursday', 1900, 2100) # Should NOT conflict

conflicts_1 = check_for_conflict(rehearsal_1, parsed_constraints)
print(f"Checking Rehearsal 1 (Wednesday): Found conflicts: {conflicts_1}")

conflicts_2 = check_for_conflict(rehearsal_2, parsed_constraints)
print(f"Checking Rehearsal 2 (Thursday): Found conflicts: {conflicts_2}")

In [ ]:
# Assuming all the classes (RehearsalSlot, constraints) and the transformer are defined above...

# --- Main Execution Logic ---
try:
    # 1. Instantiate the parser with the new grammar and transformer
    constraint_parser = Lark(constraint_grammar, start='start', parser='lalr', transformer=ConstraintTransformer())

    # 2. Define a complex unavailability string
    dancer_unavailability_string = "mo until 12, w 2-4, fri after 5pm, sunday"

    # 3. Parse the string
    parsed_constraints = constraint_parser.parse(dancer_unavailability_string)

    print(f"Input string: '{dancer_unavailability_string}'")
    print("Parsed constraint objects:")
    for c in parsed_constraints:
        print(f"  - {c}")
    print("\n")
    
    # 4. Define rehearsals to check
    rehearsal_ok = RehearsalSlot(date(2024, 9, 2), 'monday', 1300, 1500) # OK, it's after 1200
    rehearsal_conflict_1 = RehearsalSlot(date(2024, 9, 2), 'monday', 1100, 1230) # CONFLICT, overlaps with "until 1200"
    rehearsal_conflict_2 = RehearsalSlot(date(2024, 9, 4), 'wednesday', 1500, 1700) # CONFLICT, overlaps with 1400-1600
    rehearsal_conflict_3 = RehearsalSlot(date(2024, 9, 8), 'sunday', 1000, 1100) # CONFLICT, unavailable all day Sunday

    # 5. Check for conflicts
    print(f"Checking {rehearsal_ok.day_of_week} at {rehearsal_ok.start_time}:")
    print(f"  -> Conflicts: {check_for_conflict(rehearsal_ok, parsed_constraints)}\n")

    print(f"Checking {rehearsal_conflict_1.day_of_week} at {rehearsal_conflict_1.start_time}:")
    print(f"  -> Conflicts: {check_for_conflict(rehearsal_conflict_1, parsed_constraints)}\n")
    
    print(f"Checking {rehearsal_conflict_2.day_of_week} at {rehearsal_conflict_2.start_time}:")
    print(f"  -> Conflicts: {check_for_conflict(rehearsal_conflict_2, parsed_constraints)}\n")

    print(f"Checking {rehearsal_conflict_3.day_of_week} at {rehearsal_conflict_3.start_time}:")
    print(f"  -> Conflicts: {check_for_conflict(rehearsal_conflict_3, parsed_constraints)}\n")

    # 6. Test the validation
    print("Testing invalid input 'th after 25'...")
    invalid_string = "th after 25"
    constraint_parser.parse(invalid_string)

except Exception as e:
    print(f"Successfully caught expected error: {e}")

In [ ]:
# Assuming 'constraint_parser' is your Lark instance with the ConstraintTransformer

# A string with a syntactic error ("zulu") and a semantic error ("th after 99")
test_text_multiple_errors = "Monday, zulu, th after 99, wed"

print(f"Checking: '{test_text_multiple_errors}'")
ok, constraints, errors = validate_and_parse_all_constraints(constraint_parser, test_text_multiple_errors)

if ok:
    print("\nSuccess! Parsed constraints:")
    for c in constraints:
        print(f"  - {c}")
else:
    print(f"\nFailure! Reasons:\n{errors}")

print("-" * 20)

In [ ]:
# A fully valid string for comparison
test_text_valid = "mo until 12, w 2-4, fri after 5pm"
print(f"Checking: '{test_text_valid}'")
ok, constraints, errors = validate_and_parse_all_constraints(constraint_parser, test_text_valid)

if ok:
    print("\nSuccess! Parsed constraints:")
    for c in constraints:
        print(f"  - {c}")
else:
    print(f"\nFailure! Reasons:\n{errors}")

In [ ]:
# A string with a syntactic error ("zulu") and a semantic error ("th after 99")
test_text_multiple_errors = "Monday, zulu, th after 99, th after 5, wed"

print(f"Checking: '{test_text_multiple_errors}'")
ok, constraints, errors = validate_and_parse_all_constraints(constraint_parser, test_text_multiple_errors)

if ok:
    print("\nSuccess! Parsed constraints:")
    # Assuming your constraints have a __str__ or __repr__ method
    for c in constraints:
        print(f"  - {c}")
else:
    print(f"\nFailure! Reasons:\n{errors}")

# Notes on EBNF

## The Jupyternaut Prescription for Lark Transformers

Here is a concise guide with minimal examples to serve as your mental model. Think of this as the "Standard Model" for connecting your grammar to your transformer.

The fundamental principle is: **For every possible path a rule can take, there must be a corresponding method in the transformer to handle it.**

### Case 1: The Default Rule (No Alias)

This is the simplest case. The method name in your transformer **must exactly match** the rule name in your grammar.

*   **Grammar Snippet (EBNF):**
    ```lark
    time_range: time "-" time
    ```
*   **How to Identify:** A simple rule name (`time_range`) defined with a colon (`:`).
*   **Transformer Snippet (Python):**
    ```python
    class MyTransformer(Transformer):
        # Method name matches the rule name
        def time_range(self, children):
            # children will be a list of the processed parts, e.g., [900, 1700]
            start_time, end_time = children
            return (start_time, end_time)
    ```

### Case 2: The Alias Rule (The `->` Arrow)

The alias (`->`) completely overrides the default behavior. Lark will ignore the rule name and look for a method matching the alias name.

*   **Grammar Snippet (EBNF):**
    ```lark
    time_range: time "-" time -> build_explicit_range
    ```
*   **How to Identify:** Look for the arrow `->` followed by a new name (`build_explicit_range`).
*   **Transformer Snippet (Python):**
    ```python
    class MyTransformer(Transformer):
        # Method name matches the ALIAS name
        def build_explicit_range(self, children):
            start_time, end_time = children
            return (start_time, end_time)
    ```

### Case 3: The "Or" Rule (Multiple Branches)

This is the one that caught you, and it's just a combination of the first two cases. You must treat each branch (separated by a `|`) as its own independent path.

*   **Grammar Snippet (EBNF):**
    ```lark
    range: time "-" time -> build_explicit_range  // Path 1 (Alias)
         | "until"i time                        // Path 2 (Default)
    ```
*   **How to Identify:** Look for the vertical bar `|` separating different patterns. Check each pattern for an alias.
*   **Transformer Snippet (Python):** You need one method for each path.
    ```python
    class MyTransformer(Transformer):
        # Method for Path 1, matching the alias
        def build_explicit_range(self, children):
            start_time, end_time = children
            return (start_time, end_time)

        # Method for Path 2, matching the RULE name ("range")
        def range(self, children):
            # This branch only has one child: the processed `time`
            end_time = children[0]
            return (0, end_time) # from start of day until the time
    ```

### Case 4: Transforming Terminals (The Uppercase Tokens)

You can also write methods for the "atoms" of your grammar—the Terminals (usually in `ALL_CAPS`). This is extremely useful for converting raw text into useful data types like numbers or dates.

*   **Grammar Snippet (EBNF):**
    ```lark
    INT: /[0-9]+/
    ```
*   **How to Identify:** A token name in all caps.
*   **Transformer Snippet (Python):**
    ```python
    class MyTransformer(Transformer):
        # Method name matches the Terminal name
        def INT(self, number_token):
            # The argument is the token object itself
            # We convert its string value to a Python integer
            return int(number_token.value)
    ```